In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import string

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import f1_score, classification_report

from sklearn import svm
import gensim.models as g
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import wordnet

In [7]:
df = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/preprocessed_cleaned_body.csv')

In [8]:
df

,Body,Bias,cleaned_body
0,Abortion rights advocates have asked the U.S. ...,1.67,abortion right advocate asked u supreme court ...
1,A federal appeals court rejected the most dire...,0.67,federal appeal court rejected direct constitut...
2,As part of the Trump administration's effort t...,-2.75,part trump administration effort slow migrant ...
3,"President Donald Trump and ""the Trump of the T...",-4.33,president donald trump trump tropic brazilian ...
4,"U.S Senator Elizabeth Warren, who is competing...",-10.00,u senator elizabeth warren competing democrati...
...,...,...,...
1670,Only hours after the first flames that eventua...,19.75,hour first flame eventually would consume notr...
1671,Washington watchdog Judicial Watch believes th...,30.50,washington watchdog judicial watch belief fbi ...
1672,"Two Fox News personalities, as Notre Dame Cath...",29.75,two fox news personality notre dame cathedral ...
1673,Chase Bank is shutting down accounts of people...,22.00,chase bank shutting account people organizatio...


In [9]:
#extra preprocessing of data

wordsToCut = set(stopwords.words('english'))

def getPOS(words):
    #return nltk.pos_tag(word)
    return [(word, get_wordnet_pos(word)) for word in words]

def getIndividualPOS(word):
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV} 
    tag = nltk.pos_tag(word[0].upper())
    return tag_dict.get(tag)

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    return tag_dict.get(tag, wordnet.NOUN)

for index, row in df.iterrows():
    
    #removing stopwords and punctuation
    tokenized = nltk.word_tokenize(row['Body'])
    temp = [word for word in tokenized if word.lower() not in wordsToCut and word not in string.punctuation and word.isalpha()]
    
    #row['Body'] = ' '.join(temp)
    
    #row['Body'] = [word for word in nltk.word_tokenize(row['Body']) if word not in wordsToCut]
    #row['Body'] = row['Body'].lower()
    #nopunc = [char for char in row['Body'] if char not in string.punctuation]
    #nopunc.join('')
    
    #lemmatizing, ONLY KEEPING ENTITIES
    lemmatizer = WordNetLemmatizer()
    partsOfSpeech = getPOS(temp)
    
    lemmatized = [lemmatizer.lemmatize(word, pos) for word, pos in partsOfSpeech if pos == 'n']
    final = ' '.join(lemmatized)
    df.at[index, 'Body'] = final

In [10]:
right = df.loc[df['Bias'] > 0]
left = df.loc[df['Bias'] <= 0]

# 1 = Right bias, 0 = Left bias
right['Bias'] = 1
left['Bias'] = 0

cat_df = pd.concat([right, left])
cat_df

/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Body,Bias,cleaned_body
0,Abortion right advocate Supreme Court strike L...,1,abortion right advocate asked u supreme court ...
1,appeal court challenge counsel Robert Mueller ...,1,federal appeal court rejected direct constitut...
7,Michael Cohen New York per decision Department...,1,michael cohen ha officially disbarred new york...
13,President Donald Trump vetoed resolution stop ...,1,president donald trump ha vetoed resolution st...
15,body camera footage reveals police man authori...,1,newlyreleased body camera footage reveals dram...
...,...,...,...
1657,DAY BIGGEST NIGH Thursday release whatever bul...,0,day biggest coverup nigh thursday anticipating...
1658,man lady letter internet tradition online web ...,0,man lady letter also versed internet tradition...
1659,Attorney General William Barr would like remin...,0,attorney general william barr would like remin...
1661,PASTEMAGAZINE Monday night Bernie Sanders town...,0,pastemagazine — monday night bernie sander tow...


In [11]:
X = cat_df.iloc[:, cat_df.columns != 'Bias']
Y = cat_df['Bias']

In [12]:
X

,Body,cleaned_body
0,Abortion right advocate Supreme Court strike L...,abortion right advocate asked u supreme court ...
1,appeal court challenge counsel Robert Mueller ...,federal appeal court rejected direct constitut...
7,Michael Cohen New York per decision Department...,michael cohen ha officially disbarred new york...
13,President Donald Trump vetoed resolution stop ...,president donald trump ha vetoed resolution st...
15,body camera footage reveals police man authori...,newlyreleased body camera footage reveals dram...
...,...,...
1657,DAY BIGGEST NIGH Thursday release whatever bul...,day biggest coverup nigh thursday anticipating...
1658,man lady letter internet tradition online web ...,man lady letter also versed internet tradition...
1659,Attorney General William Barr would like remin...,attorney general william barr would like remin...
1661,PASTEMAGAZINE Monday night Bernie Sanders town...,pastemagazine — monday night bernie sander tow...


In [13]:
Y

0       1
1       1
7       1
13      1
15      1
       ..
1657    0
1658    0
1659    0
1661    0
1668    0
Name: Bias, Length: 1675, dtype: int64

In [14]:
bow_transformer = CountVectorizer(max_features = 800)
bow = bow_transformer.fit_transform(X['cleaned_body'])

X = pd.DataFrame(bow.toarray(), columns=bow_transformer.get_feature_names())
X

,ability,able,abortion,abuse,access,according,account,accused,across,act,...,would,wrong,wrote,year,yearold,yet,york,you,young,zealand
0,0,0,11,0,2,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,3,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,3,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
1671,0,0,0,0,0,0,0,0,0,0,...,3,1,0,0,0,0,0,0,0,0
1672,0,0,0,1,0,0,0,0,0,1,...,4,0,1,2,0,0,1,0,0,0
1673,0,1,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0


In [15]:
col_names = X.columns

scaler = MinMaxScaler()
scaled = scaler.fit_transform(X)
X = pd.DataFrame(scaled, columns=col_names)

In [16]:
X

,ability,able,abortion,abuse,access,according,account,accused,across,act,...,would,wrong,wrote,year,yearold,yet,york,you,young,zealand
0,0.0,0.00,0.52381,0.000000,0.2,0.000000,0.0,0.0,0.142857,0.000000,...,0.018182,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.153846,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
2,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.018182,0.000000,0.000000,0.000000,0.0,0.0,0.130435,0.000000,0.000000,0.0
3,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.058824,...,0.000000,0.000000,0.000000,0.076923,0.0,0.0,0.000000,0.000000,0.000000,0.0
4,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.025641,0.4,0.0,0.000000,0.066667,0.111111,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.054545,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1671,0.0,0.00,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.054545,0.166667,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
1672,0.0,0.00,0.00000,0.071429,0.0,0.000000,0.0,0.0,0.000000,0.058824,...,0.072727,0.000000,0.076923,0.051282,0.0,0.0,0.043478,0.000000,0.000000,0.0
1673,0.0,0.25,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.036364,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)

In [18]:
model = svm.SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print ("accuracy score: ", accuracy_score(y_test, y_pred))

accuracy score:  0.7136038186157518


In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
X_train

In [ ]:
y_train

### TfIdf 

In [ ]:
'''vectorizer = TfidfVectorier(use_idf=True, stop_words='english')
matrix = vectorizer.fit_transform()'''
X_tfidf = cat_df['cleaned_body']
Y_tfidf = cat_df['Bias']

In [ ]:
vectorizer = TfidfVectorizer(use_idf = True, stop_words = 'english')

In [ ]:
matrix = vectorizer.fit_transform(X_tfidf)

In [ ]:
df = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names())
#X = pd.DataFrame(bow.toarray(), columns=bow_transformer.get_feature_names())
df

In [ ]:
col_names = df.columns

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
df = pd.DataFrame(scaled, columns=col_names)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(df, Y_tfidf, random_state=1)

In [ ]:
model = svm.SVC()
print ("hi")
model.fit(X_train2, y_train2)
print ("2")
y_pred = model.predict(X_test2)
print ("3")
print ("accuracy score: ", accuracy_score(y_test2, y_pred))

### SVM with Doc2Vec

In [ ]:
cat_df

In [ ]:
text = cat_df['cleaned_body']
new_df = cat_df.drop('Body', axis=1)
#ls = [body.split() for body in text]
new_df

In [ ]:
#ls = [body.split(), rating for body, rating in new_df]

In [ ]:
#newX_train, newX_test, newy_train, newy_test = train_test_split(X, Y, random_state=1)

In [ ]:
cat_df

In [ ]:
documents = [TaggedDocument(doc.split(), [i]) for doc in cat_df['cleaned_body'] for i in cat_df['Bias']]

In [ ]:
documents

In [ ]:
pred = model.infer_vector(ls[0])